### Test Inference Yolo V4 Tiny using OpenCV DNN

- Run this notebook in your vscode

- Downgrade OpenCV to version 4.7.x
    - This downgrade is required due to Issue related to ONNX YoloV8 in OpenCV DNN for version >= 4.8.x
        - https://github.com/ultralytics/ultralytics/issues/1836
    - Open Anaconda Prompt 
    - Activate environment
    ```
    conda activate BelajarOpenCV
    ```
    - Downgrade to OpenCV 4.7.x
    ```
    pip install --force-reinstall opencv-python==4.7.0.72 --user
    ```


In [ ]:
import cv2 
import numpy as np
import utils 
utility = utils.Utils()

cv2.__version__

In [2]:
# define class names, remember in notebook 4.2 we are training yolo v4 tiny with single class `scissors`
classNames = ["scissors"]

- put the downloaded `.onnx` file from notebook 5.2 into `model/` directory,

In [9]:
# load trained Yolo V8 ONNX
model = "model/yolov8.onnx"
net = cv2.dnn.readNetFromONNX(model)


# load sample image & convert to blob
img = cv2.imread("image4.jpg")
resize_h, resize_w = 320, 320 
blob = cv2.dnn.blobFromImage(img, 1/255.0, (resize_w, resize_h), (0, 0, 0), swapRB=True, crop=False)


# do forward pass (inferencing)
net.setInput(blob)
output = net.forward()


# do a postprocessing
img = utility.postprocess_onnx(output, img, classNames, confThreshold = 0.7, nmsThreshold = 0.2, font_size=0.5, 
                        color=(255,127,0), text_color=(255,255,255), input_size=[resize_h, resize_w])


# show result 
cv2.imshow("detection result", img)
cv2.waitKey()
cv2.destroyAllWindows()

- Inference Yolo V4 Tiny From camera

In [10]:
# load trained Yolo V8 ONNX
model = "model/yolov8.onnx"
net = cv2.dnn.readNetFromONNX(model)

# load video
cap = cv2.VideoCapture(0)
           
# iterate for each frame in video
while cap.isOpened():
    
    # get image on each frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # convert to blob
    resize_h, resize_w = 320, 320 # use smaller image size to reduce computation 
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (resize_w, resize_h), (0, 0, 0), swapRB=True, crop=True)

    # do a net forward (inferencing)
    net.setInput(blob)
    output = net.forward()

    # do a postprocessing
    frame = utility.postprocess_onnx(output, frame, classNames, confThreshold = 0.5, nmsThreshold = 0.4, font_size=0.5, 
                        color=(255,127,0), text_color=(255,255,255), input_size=[resize_h, resize_w])

    # show result
    cv2.imshow('Frame',frame)

    # wait 25ms per frame and close using 'q' 
    if cv2.waitKey(1) == ord('q'):
          break


cap.release()
cv2.destroyAllWindows()